# Hadoop and MapReduce

## Пара слов про формат работы с кластером

Яндекс.Облако предлагает сервис Data Sphere как удобный способ работы с ноутбуками и в том числе с кластером. Это действительно удобно, если для вам уже все-все приготовили, однако мы с вами сейчас в позиции Full Stack Data Engineer и поэтому нам ноутбука из Data Sphere будет не достаточно - нам потребуется доступ до головной машины, чтобы исполнять bash команды на ней.

Это можно делать в простом формате - вы открываете терминал и браузер одновременно и переключаетесь между ними по мере необходимости.

Я же предлагаю сделать более удобное "рабочее место", потратив 5 минут на его настройку. Организуем себе доступ до Jupyter прямо с головной машины!

#### Организуем запуск Jupyter на головной машине

Подключимся и установим сам Jupyter.

```bash
ssh lsml-head
pip3 install notebook
```

Кластер мы будем время от времени включать/выключать, поэтому сразу настроим Jupyter на то, чтобы он включался при старте машины. Для этого воспользуемся инструментом `systemd`.

Для этого создаем файл в специальной директории с описанием параметров запуска

```bash 
sudo nano /etc/systemd/system/jupyter.service
```

Вставляем туда нашу конфигурацию

```
[Unit]
After=network.service

[Service]
ExecStart=/opt/conda/bin/jupyter notebook --no-browser --ip 0.0.0.0 --port 8888 --NotebookApp.token="" --notebook-dir=/home/ubuntu
User=ubuntu

[Install]
WantedBy=default.target
```

`Ctrl+O`, `Ctrl+X` для того чтобы сохранить и выйти из `nano`

Проверим, что точно записалось - `cat /etc/systemd/system/jupyter.service`

Активируем и стартуем наш сервис

```bash
sudo systemctl enable jupyter.service
sudo systemctl start jupyter.service
```

Отлично, Jupyter стартанул! Можно заметить, что мы запустили его без всякой авторизации, но это нормально, потому что головная машина не торчит в интернет (по этой причине у нас собственно есть прокси-машина).

Осталось научиться до нее добираться из браузера. Для этого нам опять потребуется наша прокси-машина.

Следующей командой мы создадим прокси от нашего локального порта `9999` до порта `8888` головной машины, где собственно крутится наш жупитер.

```bash
ssh -N yc-user@84.201.156.102 -L 9999:rc1a-dataproc-m-f5opzfioh9mln7bi.mdb.yandexcloud.net:8888
```

Пробуем открыть `http://localhost:9999/` - вуаля!

И еще один небольшой штрих для удобства - вместо этой громоздкой команды, можно просто подправить конфиг `~/.ssh/config`:

```
...

Host lsml-proxy
    HostName 84.201.156.102
    User yc-user
    IdentityFile ~/.ssh/id_ed25519
    LocalForward 9999 rc1a-dataproc-m-f5opzfioh9mln7bi.mdb.yandexcloud.net:8888
```

И тогда для запуска нужно всего лишь

```bash
ssh -N lsml-proxy
```

Поздравляю, если вы справились с этой настройкой, то теперь вы начинаете представлять, что же такое настоящий дата-инжиниринг :) 

Дальше я предполагаю, что я закинул этот ноутбук на головную машину и запускаю команды именно оттуда

In [1]:
! pwd

/home/ubuntu


## Основы MapReduce

В своей сути MapRedcue это очень простая парадигма. Допустим у нас есть датасет

In [2]:
! curl https://raw.githubusercontent.com/fivethirtyeight/russian-troll-tweets/master/IRAhandle_tweets_1.csv > tweets_1.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 89.9M  100 89.9M    0     0  16.8M      0  0:00:05  0:00:05 --:--:-- 21.7M


In [3]:
! head -n 2 tweets_1.csv

Мы хотим в этом датасете что-нибудь найти. Например (сюрприз-сюрприз), посчитать количество уникальных слов. Мы могли бы сделать что-то такое:

#### Вариант 1 
Используем исключительно питон и наивный алгоритм

In [4]:
%%time

from collections import Counter
import csv
import re
import sys

counter = Counter()
pattern = re.compile(r"[a-z]+")

with open('tweets_1.csv', 'r') as f:
    reader = csv.reader(f, delimiter=',')
    for row in reader:
        content = row[2]
        for match in pattern.finditer(content.lower()):
            word = match.group(0)
            counter[word] += 1

for word, count in counter.most_common(10):
    print(f"{word}\t{count}")

t	268703
co	250375
https	221366
the	69350
to	55972
a	43420
in	37099
s	36085
of	33579
http	28661
CPU times: user 4.88 s, sys: 88.8 ms, total: 4.96 s
Wall time: 4.96 s


Такое сработает только если у нас не очень много данных и они все вмещаются в оперативную память

In [5]:
! du -h tweets_1.csv

91M	tweets_1.csv


#### Вариант 2
Используем парадигму Map Reduce

В этом примере у нас всего 90 мегабайт данных, и на моем компьютере они обрабатываются за примерно 30 секунд с помощью питона. Теперь представим (это достаточно несложно), что у нас приходит новых данных приходит _десятки терабайт_ в сутки. Такое уже не поместится ни в один сервер, поэтому нам нужно придумать что-нибудь похитрее.

MapReduce как раз является парадигмой, помогающей обрабатывать большие объемы данных, за счет простоты своего устройства.

Приятная новость - для того, чтобы понять и научиться программировать программы в парадигме MapReduce вам потребуется... **5 секунд!**

<img src="https://raw.githubusercontent.com/ADKosm/lsml-2021-public/main/imgs/you-know-mapreduce.png" width="400">

Все потому что вы уже прошли семинар по Bash и научились составлять большие программы в виде компоновки небольших  программ, соединенных пайпами. По своей сути программа на MapReduce - это хорошо отмасштабированная программа вида

```bash
cat data.txt | map | sort | reduce
```

Сортировку за вас выполняет сам фреймворк (и ее вы можете дополнительно настроить точно такое как и команду sort). А также он сам разбивает данные на части и параллельно запускает операции map и reduce. 

Таким образом на самом деле Hadoop - это всего лишь гигантская машина сортировки, которая дополнительно дает вам некоторые гарантии:

* Для всех данных параллельно будет применена операция map
* Данные будут отсортированы по указанному вами ключу
* Каждый ключ будет целиком передан на один и только один reduce

Программисту остается реализовать программу, которая состоит из двух компонент: `map` и `reduce`. 

Операция `map` -- это просто функция из одного элемента в другой элемент, у которого есть первичный ключ. 

Операция `reduce` -- это коммутативная и ассоциативная агрегация всех элементов по ключу. Чтобы эти операции совершить, надо разбить весь вход на куски данных и отправить их на машины, чтобы они выполнялись в параллель, а весь выход операции map идёт в операцию shuffle, которая по одним и тем же ключам определяет записи на одинаковые хосты. 

В итоге получается, что мы можем спокойно увеличивать количество worker'ов для map операций и с увеличением количества данных мы лишь будем линейно утилизировать количество машин, то же самое с операцией reduce -- мы можем добавлять машины с ростом увеличения количества ключей линейно, не боясь того, что мы не можем позволить на одной какой-то машине больше памяти или диска.

Давайте напишем маппер и редьюсер на питоне для этой задачи:

In [17]:
! sudo pip3 install tqdm

     |████████████████████████████████| 76 kB 1.2 MB/s eta 0:00:01


In [11]:
%%writefile wordcount.py
import sys
import csv
import re


def mapper():
    pattern = re.compile(r"[a-z]+")
    for row in csv.reader(iter(sys.stdin.readline, '')):
        content = row[2]
        for match in pattern.finditer(content.lower()):
            word = match.group(0)
            print("{}\t{}".format(word, 1))


def reducer():
    word, number = next(sys.stdin).split('\t')
    number = int(number)
    for line in sys.stdin:
        current_word, current_number = line.split('\t')
        current_number = int(current_number)
        if current_word != word:
            print("{}\t{}".format(word, number))
            word = current_word
            number = current_number
        else:
            number += current_number
    print("{}\t{}".format(word, number))


if __name__ == '__main__':
    mr_command = sys.argv[1]
    {
        'map': mapper,
        'reduce': reducer
    }[mr_command]()

Overwriting wordcount.py


Важно еще удалить голову у таблицы, иначе подсчеты могут быть некоректными

In [7]:
! sed -i -e '1'd tweets_1.csv

In [8]:
! cat tweets_1.csv | python wordcount.py map | sort -k1,1 | head

a	1
a	1
a	1
a	1
a	1
a	1
a	1
a	1
a	1
a	1
sort: write failed: 'standard output': Broken pipe
sort: write error


In [18]:
%%time

! cat tweets_1.csv | \
    tqdm --total $(cat tweets_1.csv | wc -l)| \
    python wordcount.py map | \
    sort -k1,1 | \
    python wordcount.py reduce > result.txt

100%|████████████████████████████████| 243891/243891 [00:09<00:00, 26536.92it/s]
CPU times: user 211 ms, sys: 29.1 ms, total: 240 ms
Wall time: 12.7 s


In [19]:
! head result.txt

a	43420
aa	151
aaa	13
aaaaaa	1
aaaaaaaaaaaaand	1
aaaaaaaaaall	1
aaaaaaaamen	1
aaaaaaaand	2
aaaaaaargh	1
aaaaaand	2


Отлично! Слова есть, осталось только найти top-10.

In [20]:
%%writefile top10.py
import sys


def _rewind_stream(stream):
    for _ in stream:
        pass


def mapper():
    for row in sys.stdin:
        key, value = row.split('\t')
        print("{}+{}\t".format(key, value.strip()))


def reducer():
    for _ in range(10):
        key, _ = next(sys.stdin).split('\t')
        word, count = key.split("+")
        print("{}\t{}".format(word, count))
    _rewind_stream(sys.stdin)

if __name__ == '__main__':
    mr_command = sys.argv[1]
    {
        'map': mapper,
        'reduce': reducer
    }[mr_command]()

Writing top10.py


In [21]:
! cat result.txt | \
    tqdm --total $(cat result.txt | wc -l) | \
    python top10.py map | \
    sort -t'+' -k2,2nr -k1,1 | \
    python top10.py reduce > top-10.txt

100%|███████████████████████████████| 346613/346613 [00:00<00:00, 615506.93it/s]


In [22]:
! cat top-10.txt

t	268703
co	250375
https	221366
the	69350
to	55972
a	43420
in	37099
s	36085
of	33579
http	28661


На MapReduce мы задачу переписали, однако быстрее работать она пока не стала. Все дело в том, что мы это еще не на кластере запускали! Время запускать все на настоящем кластере!

### Загружаем данные в HDFS

При работе с HDFS нужно понимать, что есть два места, где хранятся данные

1. На локальных жестких дисках машин кластера - это деволтная система, на нее можно посмотреть через `hdfs dfs -ls /`

2. В Object Storage - для работы с ней, нужно указывать путь до бакета - `hdfs dfs -ls s3a://lsml2022alexius/`

In [23]:
! curl -O https://raw.githubusercontent.com/fivethirtyeight/russian-troll-tweets/master/IRAhandle_tweets_{`seq -s , 1 13`}.csv


[1/13]: https://raw.githubusercontent.com/fivethirtyeight/russian-troll-tweets/master/IRAhandle_tweets_1.csv --> IRAhandle_tweets_1.csv
--_curl_--https://raw.githubusercontent.com/fivethirtyeight/russian-troll-tweets/master/IRAhandle_tweets_1.csv
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 89.9M  100 89.9M    0     0  50.5M      0  0:00:01  0:00:01 --:--:-- 50.5M

[2/13]: https://raw.githubusercontent.com/fivethirtyeight/russian-troll-tweets/master/IRAhandle_tweets_2.csv --> IRAhandle_tweets_2.csv
--_curl_--https://raw.githubusercontent.com/fivethirtyeight/russian-troll-tweets/master/IRAhandle_tweets_2.csv
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 90.0M  100 90.0M    0     0  13.5M      0  0:00:06  0:00:06 --:--:-- 19.3M

[3/13]: https://raw.githubuse

Подформатируем

In [24]:
! for i in {1..13}; do sed IRAhandle_tweets_$i.csv -i -e '1'd && echo "Finish $i" ; done

Finish 1
Finish 2
Finish 3
Finish 4
Finish 5
Finish 6
Finish 7
Finish 8
Finish 9
Finish 10
Finish 11
Finish 12
Finish 13


Создадим отдельную папку для этих данных в HDFS

In [25]:
! hdfs dfs -ls /

Found 4 items
drwx------   - mapred hadoop          0 2022-01-24 09:44 /hadoop
drwxrwxrwt   - hdfs   hadoop          0 2022-01-24 09:43 /tmp
drwxrwxrwt   - hdfs   hadoop          0 2022-01-30 10:44 /user
drwxrwxrwt   - hdfs   hadoop          0 2022-01-24 09:44 /var


In [27]:
! hdfs dfs -rm -r /user/tweets/data
! hdfs dfs -mkdir -p /user/tweets/data

rm: `/user/tweets/data': No such file or directory


Note: все команды для hdfs смотреть здесь - https://hadoop.apache.org/docs/current/hadoop-project-dist/hadoop-common/FileSystemShell.html

Заливаем данные

In [28]:
! hdfs dfs -put IRAhandle_tweets_* /user/tweets/data/

In [41]:
! sudo chmod 0777 /usr/lib/hadoop/logs
! sudo -u hdfs hdfs balancer 

2022-01-30 12:33:19,998 INFO balancer.Balancer: namenodes  = [hdfs://rc1a-dataproc-m-mk7wm6vzosv08ggu.mdb.yandexcloud.net:8020]
2022-01-30 12:33:20,005 INFO balancer.Balancer: parameters = Balancer.BalancerParameters [BalancingPolicy.Node, threshold = 10.0, max idle iteration = 5, #excluded nodes = 0, #included nodes = 0, #source nodes = 0, #blockpools = 0, run during upgrade = false]
2022-01-30 12:33:20,005 INFO balancer.Balancer: included nodes = []
2022-01-30 12:33:20,005 INFO balancer.Balancer: excluded nodes = []
2022-01-30 12:33:20,005 INFO balancer.Balancer: source nodes = []
Time Stamp               Iteration#  Bytes Already Moved  Bytes Left To Move  Bytes Being Moved  NameNode
2022-01-30 12:33:20,010 INFO balancer.NameNodeConnector: getBlocks calls for hdfs://rc1a-dataproc-m-mk7wm6vzosv08ggu.mdb.yandexcloud.net:8020 will be rate-limited to 20 per second
2022-01-30 12:33:21,104 INFO balancer.Balancer: dfs.namenode.get-blocks.max-qps = 20 (default=20)
2022-01-30 12:33:21,104 IN

In [29]:
! hdfs dfs -ls /user/tweets/data/

Found 13 items
-rw-r--r--   1 ubuntu hadoop   94371561 2022-01-30 12:13 /user/tweets/data/IRAhandle_tweets_1.csv
-rw-r--r--   1 ubuntu hadoop   94371615 2022-01-30 12:13 /user/tweets/data/IRAhandle_tweets_10.csv
-rw-r--r--   1 ubuntu hadoop   94371552 2022-01-30 12:13 /user/tweets/data/IRAhandle_tweets_11.csv
-rw-r--r--   1 ubuntu hadoop   94371703 2022-01-30 12:13 /user/tweets/data/IRAhandle_tweets_12.csv
-rw-r--r--   1 ubuntu hadoop    8238864 2022-01-30 12:13 /user/tweets/data/IRAhandle_tweets_13.csv
-rw-r--r--   1 ubuntu hadoop   94371748 2022-01-30 12:13 /user/tweets/data/IRAhandle_tweets_2.csv
-rw-r--r--   1 ubuntu hadoop   94371796 2022-01-30 12:13 /user/tweets/data/IRAhandle_tweets_3.csv
-rw-r--r--   1 ubuntu hadoop   94371606 2022-01-30 12:13 /user/tweets/data/IRAhandle_tweets_4.csv
-rw-r--r--   1 ubuntu hadoop   94371616 2022-01-30 12:13 /user/tweets/data/IRAhandle_tweets_5.csv
-rw-r--r--   1 ubuntu hadoop   94371646 2022-01-30 12:13 /user/tweets/data/IRAhandle_tweets_6.csv
-

### Запускаем MapReduce

Проверяем, что скрипты на головной машине

In [30]:
! cat wordcount.py

import sys
import csv
import re


def mapper():
    pattern = re.compile(r"[a-z]+")
    for row in csv.reader(iter(sys.stdin.readline, '')):
        content = row[2]
        for match in pattern.finditer(content.lower()):
            word = match.group(0)
            print("{}\t{}".format(word, 1))


def reducer():
    word, number = next(sys.stdin).split('\t')
    number = int(number)
    for line in sys.stdin:
        current_word, current_number = line.split('\t')
        current_number = int(current_number)
        if current_word != word:
            print("{}\t{}".format(word, number))
            word = current_word
            number = current_number
        else:
            number += current_number
    print("{}\t{}".format(word, number))


if __name__ == '__main__':
    mr_command = sys.argv[1]
    {
        'map': mapper,
        'reduce': reducer
    }[mr_command]()


In [31]:
! cat top10.py

import sys


def _rewind_stream(stream):
    for _ in stream:
        pass


def mapper():
    for row in sys.stdin:
        key, value = row.split('\t')
        print("{}+{}\t".format(key, value.strip()))


def reducer():
    for _ in range(10):
        key, _ = next(sys.stdin).split('\t')
        word, count = key.split("+")
        print("{}\t{}".format(word, count))
    _rewind_stream(sys.stdin)

if __name__ == '__main__':
    mr_command = sys.argv[1]
    {
        'map': mapper,
        'reduce': reducer
    }[mr_command]()


Собираем команду на запуск

In [32]:
! file /usr/hdp/current/hadoop-mapreduce-client/hadoop-streaming.jar

/usr/hdp/current/hadoop-mapreduce-client/hadoop-streaming.jar: cannot open `/usr/hdp/current/hadoop-mapreduce-client/hadoop-streaming.jar' (No such file or directory)


In [33]:
%%time

! hdfs dfs -rm -r /user/tweets/result || true
! yarn jar /usr/lib/hadoop-mapreduce/hadoop-streaming.jar \
-D mapreduce.job.name="word-count" \
-D mapreduce.job.reduces=3 \
-files ~/wordcount.py \
-mapper "python3 wordcount.py map" \
-reducer "python3 wordcount.py reduce" \
-input /user/tweets/data/ \
-output /user/tweets/result/

rm: `/user/tweets/result': No such file or directory
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-3.2.2.jar] /tmp/streamjob7074396873421902954.jar tmpDir=null
2022-01-30 12:21:34,703 INFO client.RMProxy: Connecting to ResourceManager at rc1a-dataproc-m-mk7wm6vzosv08ggu.mdb.yandexcloud.net/10.128.0.26:8032
2022-01-30 12:21:34,911 INFO client.AHSProxy: Connecting to Application History server at rc1a-dataproc-m-mk7wm6vzosv08ggu.mdb.yandexcloud.net/10.128.0.26:10200
2022-01-30 12:21:34,947 INFO client.RMProxy: Connecting to ResourceManager at rc1a-dataproc-m-mk7wm6vzosv08ggu.mdb.yandexcloud.net/10.128.0.26:8032
2022-01-30 12:21:34,948 INFO client.AHSProxy: Connecting to Application History server at rc1a-dataproc-m-mk7wm6vzosv08ggu.mdb.yandexcloud.net/10.128.0.26:10200
2022-01-30 12:21:35,206 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/ubuntu/.staging/job_1643538737352_0002
2022-01-30 12:21:35,923 INFO mapred.FileInputFo

Помимо того, что можно следить здесь в терминале, за выполнением можно наблюдать через UI-proxy в интерфейсе облака

Смотрим результат


In [35]:
! hdfs dfs -ls /user/tweets/result

Found 4 items
-rw-r--r--   1 ubuntu hadoop          0 2022-01-30 12:22 /user/tweets/result/_SUCCESS
-rw-r--r--   1 ubuntu hadoop   10107405 2022-01-30 12:22 /user/tweets/result/part-00000
-rw-r--r--   1 ubuntu hadoop   10134121 2022-01-30 12:22 /user/tweets/result/part-00001
-rw-r--r--   1 ubuntu hadoop   10118293 2022-01-30 12:22 /user/tweets/result/part-00002


In [36]:
! hdfs dfs -cat /user/tweets/result/part-* | head

aa	1726
aaaaa	7
aaaaaaaaaaaaaa	3
aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaannnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnddddddddddddddddddddddddddddddddddddd	1
aaaaaaaaaaaaaaaaaaah	1
aaaaaaaaaaaaand	1
aaaaaaaaannnnnnnnnnnddddddddddddd	1
aaaaaaaah	1
aaaaaaaamen	1
aaaaaaagh	2
cat: Unable to write to output stream.
cat: Unable to write to output stream.
cat: Unable to write to output stream.


Запустим вторую задачу 

In [37]:
%%time

! hdfs dfs -rm -r /user/tweets/top10/
! yarn jar /usr/lib/hadoop-mapreduce/hadoop-streaming.jar \
-D mapreduce.job.name="top-10" \
-D mapreduce.job.reduces=1 \
-D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapreduce.lib.partition.KeyFieldBasedComparator \
-D mapreduce.partition.keycomparator.options="-k2,2nr -k1,1" \
-D mapreduce.map.output.key.field.separator='+' \
-files top10.py \
-mapper "python top10.py map" \
-reducer "python top10.py reduce" \
-input /user/tweets/result/ \
-output /user/tweets/top10/

rm: `/user/tweets/top10/': No such file or directory
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-3.2.2.jar] /tmp/streamjob1782456448161600928.jar tmpDir=null
2022-01-30 12:25:30,007 INFO client.RMProxy: Connecting to ResourceManager at rc1a-dataproc-m-mk7wm6vzosv08ggu.mdb.yandexcloud.net/10.128.0.26:8032
2022-01-30 12:25:30,242 INFO client.AHSProxy: Connecting to Application History server at rc1a-dataproc-m-mk7wm6vzosv08ggu.mdb.yandexcloud.net/10.128.0.26:10200
2022-01-30 12:25:30,282 INFO client.RMProxy: Connecting to ResourceManager at rc1a-dataproc-m-mk7wm6vzosv08ggu.mdb.yandexcloud.net/10.128.0.26:8032
2022-01-30 12:25:30,284 INFO client.AHSProxy: Connecting to Application History server at rc1a-dataproc-m-mk7wm6vzosv08ggu.mdb.yandexcloud.net/10.128.0.26:10200
2022-01-30 12:25:30,515 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/ubuntu/.staging/job_1643538737352_0003
2022-01-30 12:25:31,242 INFO mapred.FileInputFo

In [42]:
! hdfs dfs -ls /user/tweets/top10

Found 2 items
-rw-r--r--   1 ubuntu hadoop          0 2022-01-30 12:26 /user/tweets/top10/_SUCCESS
-rw-r--r--   1 ubuntu hadoop        106 2022-01-30 12:26 /user/tweets/top10/part-00000


In [43]:
! hdfs dfs -cat /user/tweets/top10/part-*

t	3015051
co	2833375
https	2454132
the	591885
to	589004
in	457433
a	412888
s	397889
http	375299
of	350983


### Distributed cache

Помимо самого скрипта, мы можем положить в MapReduce любой другой файл, который может пригодиться для работы программы. Например при подсчете количества слов мы бы хотели выкинуть "стоп-слова". Их количество скорее всего не очень большое поэтому смело может передавать их обычным файлом. Hadoop гарантирует, что доставит все файлы ко всем машинам.

In [44]:
! hdfs dfs -cat /user/tweets/top10/part-* > stop-words.txt

#### Хозяйке на заметку

В питоне уже есть хорошая стандартная библиотека, которая позволяет вам гораздо удобнее работать с такимим стримовыми данными. Давайте напишем новую задачу со стоп словами, чтобы они смотрелись поприличнее.

In [45]:
%%writefile wordcount2.py

import sys
import csv
import re
from itertools import groupby


def csv_stream():
    return csv.reader(iter(sys.stdin.readline, ''))

def kv_stream(sep="\t"):
    return map(lambda x: x.split(sep), sys.stdin)


def mapper():
    pattern = re.compile(r"[a-z]+")
    for row in csv_stream():
        content = row[2]
        for match in pattern.finditer(content.lower()):
            word = match.group(0)
            print("{}\t{}".format(word, 1))


def reducer():
    for key, group in groupby(kv_stream(), lambda x: x[0]):
        word = key
        number = sum(int(x) for _, x in group)
        print("{}\t{}".format(word, number))


if __name__ == '__main__':
    mr_command = sys.argv[1]
    {
        'map': mapper,
        'reduce': reducer
    }[mr_command]()

Writing wordcount2.py


In [46]:
%%writefile top10-2.py

import sys
import collections
from itertools import islice

def build_stop_words():
    with open('stop-words.txt', 'r') as f:
        stop_words = {x.split('\t')[0] for x in f}
    return stop_words

def kv_stream(sep="\t"):
    return map(lambda x: x.split(sep), sys.stdin)

def rewind():
    collections.deque(sys.stdin, maxlen=0)

def mapper():
    for key, value in kv_stream():
        print("{}+{}\t".format(key, value.strip()))

def reducer():
    stop_words = build_stop_words()
    first_10_stream = islice(filter(lambda x: x[0] not in stop_words, kv_stream('+')), 10)
    
    for word, count in first_10_stream:
        print("{}\t{}".format(word, count.strip()))
    rewind()

if __name__ == '__main__':
    mr_command = sys.argv[1]
    {
        'map': mapper,
        'reduce': reducer
    }[mr_command]()

Writing top10-2.py


In [47]:
! cat stop-words.txt

t	3015051
co	2833375
https	2454132
the	591885
to	589004
in	457433
a	412888
s	397889
http	375299
of	350983


In [48]:
%%time

! hdfs dfs -rm -r /user/tweets/top10-stop-words/ || true
! yarn jar /usr/lib/hadoop-mapreduce/hadoop-streaming.jar \
-D mapreduce.job.name="top-10-stop-words" \
-D mapreduce.job.reduces=1 \
-D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapreduce.lib.partition.KeyFieldBasedComparator \
-D mapreduce.partition.keycomparator.options="-k2,2nr -k1,1" \
-D mapreduce.map.output.key.field.separator='+' \
-files top10-2.py,stop-words.txt \
-mapper "python top10-2.py map" \
-reducer "python top10-2.py reduce" \
-input /user/tweets/result/ \
-output /user/tweets/top10-stop-words/

rm: `/user/tweets/top10-stop-words/': No such file or directory
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-3.2.2.jar] /tmp/streamjob7790193568073565258.jar tmpDir=null
2022-01-30 12:37:32,981 INFO client.RMProxy: Connecting to ResourceManager at rc1a-dataproc-m-mk7wm6vzosv08ggu.mdb.yandexcloud.net/10.128.0.26:8032
2022-01-30 12:37:33,199 INFO client.AHSProxy: Connecting to Application History server at rc1a-dataproc-m-mk7wm6vzosv08ggu.mdb.yandexcloud.net/10.128.0.26:10200
2022-01-30 12:37:33,252 INFO client.RMProxy: Connecting to ResourceManager at rc1a-dataproc-m-mk7wm6vzosv08ggu.mdb.yandexcloud.net/10.128.0.26:8032
2022-01-30 12:37:33,255 INFO client.AHSProxy: Connecting to Application History server at rc1a-dataproc-m-mk7wm6vzosv08ggu.mdb.yandexcloud.net/10.128.0.26:10200
2022-01-30 12:37:33,467 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/ubuntu/.staging/job_1643538737352_0004
2022-01-30 12:37:34,595 INFO mapred.

In [49]:
! hdfs dfs -cat /user/tweets/top10-stop-words/*

i	287232
for	272995
and	247749
is	246856
on	210172
you	196950
trump	169520
news	156101
it	152816
with	134178


### Ускоряем вычисления 

Несмотря на все оптимизации внутри Hadoop, самое узкое место - это передача данных от mapper к reducer. Таким образом если у нас получиться ускорить выполнение .

In [50]:
%%writefile wordcount3.py

import sys
import csv
import re
from itertools import groupby
from collections import Counter


def csv_stream():
    return csv.reader(iter(sys.stdin.readline, ''))

def kv_stream(sep="\t"):
    return map(lambda x: x.split(sep), sys.stdin)


def mapper():
    counter = Counter()
    pattern = re.compile(r"[a-z]+")
    for row in csv_stream():
        content = row[2]
        for match in pattern.finditer(content.lower()):
            word = match.group(0)
            counter[word] += 1
    
    for word, number in counter.items():
        print("{}\t{}".format(word, number))


def reducer():
    for key, group in groupby(kv_stream(), lambda x: x[0]):
        word = key
        number = sum(int(x) for _, x in group)
        print("{}\t{}".format(word, number))


if __name__ == '__main__':
    mr_command = sys.argv[1]
    {
        'map': mapper,
        'reduce': reducer
    }[mr_command]()

Writing wordcount3.py


In [51]:
%%time

! hdfs dfs -rm -r /user/tweets/result-fast1 || true
! yarn jar /usr/lib/hadoop-mapreduce/hadoop-streaming.jar \
-D mapreduce.job.name="word-count" \
-D mapreduce.job.reduces=3 \
-files ~/wordcount3.py \
-mapper "python3 wordcount3.py map" \
-reducer "python3 wordcount3.py reduce" \
-input /user/tweets/data/ \
-output /user/tweets/result-fast1/

rm: `/user/tweets/result-fast1': No such file or directory
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-3.2.2.jar] /tmp/streamjob3815495773780317704.jar tmpDir=null
2022-01-30 12:40:21,324 INFO client.RMProxy: Connecting to ResourceManager at rc1a-dataproc-m-mk7wm6vzosv08ggu.mdb.yandexcloud.net/10.128.0.26:8032
2022-01-30 12:40:21,551 INFO client.AHSProxy: Connecting to Application History server at rc1a-dataproc-m-mk7wm6vzosv08ggu.mdb.yandexcloud.net/10.128.0.26:10200
2022-01-30 12:40:21,585 INFO client.RMProxy: Connecting to ResourceManager at rc1a-dataproc-m-mk7wm6vzosv08ggu.mdb.yandexcloud.net/10.128.0.26:8032
2022-01-30 12:40:21,586 INFO client.AHSProxy: Connecting to Application History server at rc1a-dataproc-m-mk7wm6vzosv08ggu.mdb.yandexcloud.net/10.128.0.26:10200
2022-01-30 12:40:21,805 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/ubuntu/.staging/job_1643538737352_0005
2022-01-30 12:40:22,949 INFO mapred.FileI

In [52]:
! hdfs dfs -cat /user/tweets/result-fast1/* | head

aa	1726
aaaaa	7
aaaaaaaaaaaaaa	3
aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaannnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnddddddddddddddddddddddddddddddddddddd	1
aaaaaaaaaaaaaaaaaaah	1
aaaaaaaaaaaaand	1
aaaaaaaaannnnnnnnnnnddddddddddddd	1
aaaaaaaah	1
aaaaaaaamen	1
aaaaaaagh	2
cat: Unable to write to output stream.
cat: Unable to write to output stream.
cat: Unable to write to output stream.


Однако у этого решения есть **очень большой минус** - сложность по памяти **O(n)**. Это означает, что вычисление может упасть если данные попадутся неудачные. 

Важный принцип работы с большими данными - все алгоритмы должны работать меньше чем за O(n). Это относится не только к MapReduce, а в целом почти к любым инструментам обработки больших данных.

In [1]:
! hdfs dfs -ls /user/

Found 3 items
drwxr-xr-x   - hive   hadoop          0 2022-01-24 09:44 /user/hive
drwxr-xr-x   - livy   hadoop          0 2022-01-30 10:44 /user/livy
drwxr-xr-x   - ubuntu hadoop          0 2022-01-30 12:47 /user/tweets


### Используем комбайнер

Чтобы побороться с этой бедой, воспользуемся дополнительным инструментом в Hadoop - Combiner. По сути это маленький Reduce, который запускается после маппера. Это позволяет уменьшить количество выходных данных с Map стадии.

<img src="https://habrastorage.org/getpro/habr/post_images/587/2d2/dfe/5872d2dfe12643665370708d225bc1d4.jpg">

In [53]:
%%time

! hdfs dfs -rm -r /user/tweets/result-fast2 || true
! yarn jar /usr/lib/hadoop-mapreduce/hadoop-streaming.jar \
-D mapreduce.job.name="word-count" \
-D mapreduce.job.reduces=3 \
-files ~/wordcount2.py \
-mapper "python3 wordcount2.py map" \
-combiner "python3 wordcount2.py reduce" \
-reducer "python3 wordcount2.py reduce" \
-input /user/tweets/data/ \
-output /user/tweets/result-fast2/

rm: `/user/tweets/result-fast2': No such file or directory
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-3.2.2.jar] /tmp/streamjob8175970712716338649.jar tmpDir=null
2022-01-30 12:43:12,573 INFO client.RMProxy: Connecting to ResourceManager at rc1a-dataproc-m-mk7wm6vzosv08ggu.mdb.yandexcloud.net/10.128.0.26:8032
2022-01-30 12:43:12,796 INFO client.AHSProxy: Connecting to Application History server at rc1a-dataproc-m-mk7wm6vzosv08ggu.mdb.yandexcloud.net/10.128.0.26:10200
2022-01-30 12:43:12,831 INFO client.RMProxy: Connecting to ResourceManager at rc1a-dataproc-m-mk7wm6vzosv08ggu.mdb.yandexcloud.net/10.128.0.26:8032
2022-01-30 12:43:12,832 INFO client.AHSProxy: Connecting to Application History server at rc1a-dataproc-m-mk7wm6vzosv08ggu.mdb.yandexcloud.net/10.128.0.26:10200
2022-01-30 12:43:13,045 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/ubuntu/.staging/job_1643538737352_0006
2022-01-30 12:43:13,774 INFO mapred.FileI

In [55]:
! hdfs dfs -cat /user/tweets/result-fast1/* | head

aa	1726
aaaaa	7
aaaaaaaaaaaaaa	3
aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaannnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnddddddddddddddddddddddddddddddddddddd	1
aaaaaaaaaaaaaaaaaaah	1
aaaaaaaaaaaaand	1
aaaaaaaaannnnnnnnnnnddddddddddddd	1
aaaaaaaah	1
aaaaaaaamen	1
aaaaaaagh	2
cat: Unable to write to output stream.
cat: Unable to write to output stream.
cat: Unable to write to output stream.


Часто combiner может просто совпадать с reducer однако это не всегда так по следующей причине - combiner не имеет права менять формат вывода после стадии map.

Hadoop самостоятельно опеределяет целесообразность запуска combiner и может его не запускать вовсе.
Или например задача может вообще не подходить под такую модель запуска. Если мы ищем среднее, то нельзя заранее подсчитывать среднее на стадии combiner - макмимум, что мы там можем запустить - это подсчет количество и суммы.

In [57]:
%%writefile top10-3.py

import sys
import collections
from itertools import islice

def build_stop_words():
    with open('stop-words.txt', 'r') as f:
        stop_words = {x.split('\t')[0] for x in f}
    return stop_words

def kv_stream(sep="\t"):
    return map(lambda x: x.split(sep), sys.stdin)

def rewind():
    collections.deque(sys.stdin, maxlen=0)

def mapper():
    for key, value in kv_stream():
        print("{}+{}\t".format(key, value.strip()))

def reducer():
    stop_words = build_stop_words()
    first_10_stream = islice(filter(lambda x: x[0] not in stop_words, kv_stream('+')), 10)
    
    for word, count in first_10_stream:
        print("{}\t{}".format(word, count.strip()))
    rewind()
    
def combiner():
    stop_words = build_stop_words()
    first_10_stream = islice(filter(lambda x: x[0] not in stop_words, kv_stream('+')), 10)
    
    for word, count in first_10_stream:
        print("{}+{}\t".format(word, count.strip()))
    rewind()

if __name__ == '__main__':
    mr_command = sys.argv[1]
    {
        'map': mapper,
        'reduce': reducer,
        'combiner': combiner
    }[mr_command]()

Overwriting top10-3.py


In [58]:
%%time

! hdfs dfs -rm -r /user/tweets/top10-fast || true
! yarn jar /usr/lib/hadoop-mapreduce/hadoop-streaming.jar \
-D mapreduce.job.name="word-count" \
-D mapreduce.job.reduces=1 \
-D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapreduce.lib.partition.KeyFieldBasedComparator \
-D mapreduce.partition.keycomparator.options="-k2,2nr -k1,1" \
-D mapreduce.map.output.key.field.separator='+' \
-files ~/top10-3.py,stop-words.txt \
-mapper "python3 top10-3.py map" \
-combiner "python3 top10-3.py combiner" \
-reducer "python3 top10-3.py reduce" \
-input /user/tweets/result-fast1 \
-output /user/tweets/top10-fast/

rm: `/user/tweets/top10-fast': No such file or directory
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-3.2.2.jar] /tmp/streamjob9119716366048105986.jar tmpDir=null
2022-01-30 12:47:15,994 INFO client.RMProxy: Connecting to ResourceManager at rc1a-dataproc-m-mk7wm6vzosv08ggu.mdb.yandexcloud.net/10.128.0.26:8032
2022-01-30 12:47:16,213 INFO client.AHSProxy: Connecting to Application History server at rc1a-dataproc-m-mk7wm6vzosv08ggu.mdb.yandexcloud.net/10.128.0.26:10200
2022-01-30 12:47:16,248 INFO client.RMProxy: Connecting to ResourceManager at rc1a-dataproc-m-mk7wm6vzosv08ggu.mdb.yandexcloud.net/10.128.0.26:8032
2022-01-30 12:47:16,249 INFO client.AHSProxy: Connecting to Application History server at rc1a-dataproc-m-mk7wm6vzosv08ggu.mdb.yandexcloud.net/10.128.0.26:10200
2022-01-30 12:47:16,445 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/ubuntu/.staging/job_1643538737352_0007
2022-01-30 12:47:17,196 INFO mapred.FileInp

In [59]:
! hdfs dfs -cat /user/tweets/top10-fast/* 

i	287232
for	272995
and	247749
is	246856
on	210172
you	196950
trump	169520
news	156101
it	152816
with	134178


### Кастомный Partitioner

В Hadoop MapReduce можно указывать свой кастомный партишенер, который будет определять, как разбивать данные по редюсерам.

Это бывает важно, когда вы используете сложный ключ и много редюсеров - вполне возможно вы захотите, чтобы такие ключи определенным образом распределялись по редюс-задачам.

Для наглядности давайте решим такую задачу - для каждого пользователя подсчитаем, на каких языках он писал твиты и в каком количестве

In [38]:
%%writefile lang-distribution.py

import sys
import csv
from collections import Counter
from itertools import groupby

def kv_stream(sep="\t"):
    return map(lambda x: x.split(sep), sys.stdin)

def csv_stream():
    return csv.reader(iter(sys.stdin.readline, ''))

def mapper():
    for row in csv_stream():
        author, lang = row[1], row[4]
        print("{}+{}\t1".format(author.strip(), lang.strip()))

def reducer():
    for author, records in groupby(kv_stream('+'), lambda x: x[0]):
        langs_stream = (x.split('\t') for _, x in records)
        for lang, group in groupby(langs_stream, lambda x: x[0]):
            count = sum(int(x) for _, x in group)
            print("{}+{}\t{}".format(author, lang, count))

if __name__ == '__main__':
    mr_command = sys.argv[1]
    {
        'map': mapper,
        'reduce': reducer,
    }[mr_command]()

Overwriting lang-distribution.py


Параметры

```
-D mapred.partitioner.class=org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner \
-D mapreduce.partition.keypartitioner.options="-k1,1" \
```

Указывают на то, что разделятся на редюсеры записи должны не по полному ключу, а только по первой его части

In [40]:
! hdfs dfs -rm -r /user/tweets/lang-dist || true
! yarn jar /usr/lib/hadoop-mapreduce/hadoop-streaming.jar \
-D mapreduce.job.name="lang-dist" \
-D mapreduce.job.reduces=3 \
-D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapreduce.lib.partition.KeyFieldBasedComparator \
-D mapreduce.partition.keycomparator.options="-k1,1 -k2,2" \
-D mapreduce.map.output.key.field.separator='+' \
-D mapred.partitioner.class=org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner \
-D mapreduce.partition.keypartitioner.options="-k1,1" \
-files ~/lang-distribution.py \
-mapper "python3 lang-distribution.py map" \
-reducer "python3 lang-distribution.py reduce" \
-input /user/tweets/data/ \
-output /user/tweets/lang-dist

Deleted /user/tweets/lang-dist
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-3.2.2.jar] /tmp/streamjob4237600768320552868.jar tmpDir=null
2022-01-30 13:56:15,431 INFO client.RMProxy: Connecting to ResourceManager at rc1a-dataproc-m-mk7wm6vzosv08ggu.mdb.yandexcloud.net/10.128.0.26:8032
2022-01-30 13:56:15,676 INFO client.AHSProxy: Connecting to Application History server at rc1a-dataproc-m-mk7wm6vzosv08ggu.mdb.yandexcloud.net/10.128.0.26:10200
2022-01-30 13:56:15,708 INFO client.RMProxy: Connecting to ResourceManager at rc1a-dataproc-m-mk7wm6vzosv08ggu.mdb.yandexcloud.net/10.128.0.26:8032
2022-01-30 13:56:15,709 INFO client.AHSProxy: Connecting to Application History server at rc1a-dataproc-m-mk7wm6vzosv08ggu.mdb.yandexcloud.net/10.128.0.26:10200
2022-01-30 13:56:15,934 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/ubuntu/.staging/job_1643547082499_0003
2022-01-30 13:56:16,305 INFO mapred.FileInputFormat: Total input file

In [41]:
! hdfs dfs -cat /user/tweets/lang-dist/* | head 

1488REASONS+Russian	50
1488REASONS+Serbian	1
1488REASONS+Ukrainian	1
1D_NICOLE_+Albanian	1
1D_NICOLE_+English	41
1D_NICOLE_+Tagalog (Filipino)	2
1ERIK_LEE+English	2
459JISALGE+Russian	1
4MYSQUAD+Arabic	5
4MYSQUAD+Catalan	1
cat: Unable to write to output stream.
cat: Unable to write to output stream.
cat: Unable to write to output stream.


### Как дебажить ошибки

Через yarn можно выводить логи приложения

In [42]:
%%writefile mistake.py

import sys
import csv
import re
from itertools import groupby


def csv_stream():
    return csv.reader(iter(sys.stdin.readline, ''))

def kv_stream(sep="\t"):
    return map(lambda x: x.split(sep), sys.stdin)


def mapper():
    pattern = re.compile(r"[a-z]+")
    for row in csv_stream():
        content = row[2]
        for match in pattern.finditer(content.lower()):
            word = match.group(0)
            strange_number = 1.0 / (len(word) - 1)
            print("{}\t{}".format(word, strange_number))


def reducer():
    for key, group in groupby(kv_stream(), lambda x: x[0]):
        word = key
        number = sum(float(x) for _, x in group)
        print("{}\t{}".format(word, number))


if __name__ == '__main__':
    mr_command = sys.argv[1]
    {
        'map': mapper,
        'reduce': reducer
    }[mr_command]()

Writing mistake.py


In [46]:
! hdfs dfs -rm -r /user/tweets/mistake || true
! yarn jar /usr/lib/hadoop-mapreduce/hadoop-streaming.jar \
-D mapreduce.job.name="mistake" \
-D mapreduce.job.reduces=3 \
-files ~/mistake.py \
-mapper "python3 mistake.py map" \
-reducer "python3 mistake.py reduce" \
-input /user/tweets/data/ \
-output /user/tweets/mistake

Deleted /user/tweets/mistake
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-3.2.2.jar] /tmp/streamjob2385587628438507101.jar tmpDir=null
2022-01-30 14:12:15,607 INFO client.RMProxy: Connecting to ResourceManager at rc1a-dataproc-m-mk7wm6vzosv08ggu.mdb.yandexcloud.net/10.128.0.26:8032
2022-01-30 14:12:15,839 INFO client.AHSProxy: Connecting to Application History server at rc1a-dataproc-m-mk7wm6vzosv08ggu.mdb.yandexcloud.net/10.128.0.26:10200
2022-01-30 14:12:15,884 INFO client.RMProxy: Connecting to ResourceManager at rc1a-dataproc-m-mk7wm6vzosv08ggu.mdb.yandexcloud.net/10.128.0.26:8032
2022-01-30 14:12:15,885 INFO client.AHSProxy: Connecting to Application History server at rc1a-dataproc-m-mk7wm6vzosv08ggu.mdb.yandexcloud.net/10.128.0.26:10200
2022-01-30 14:12:16,123 INFO mapreduce.JobResourceUploader: Disabling Erasure Coding for path: /tmp/hadoop-yarn/staging/ubuntu/.staging/job_1643547082499_0005
2022-01-30 14:12:16,862 INFO mapred.FileInputFormat: Total input files 

In [49]:
! yarn logs -applicationId application_1643547082499_0005 -log_files stderr | head -n 50

2022-01-30 14:13:23,022 INFO client.RMProxy: Connecting to ResourceManager at rc1a-dataproc-m-mk7wm6vzosv08ggu.mdb.yandexcloud.net/10.128.0.26:8032
2022-01-30 14:13:23,269 INFO client.AHSProxy: Connecting to Application History server at rc1a-dataproc-m-mk7wm6vzosv08ggu.mdb.yandexcloud.net/10.128.0.26:10200
Container: container_1643547082499_0005_01_000026 on rc1a-dataproc-d-9r5dg188d90q4f2a.mdb.yandexcloud.net_44115
LogAggregationType: AGGREGATED
LogType:stderr
LogLastModifiedTime:Sun Jan 30 14:12:53 +0000 2022
LogLength:208
LogContents:
Traceback (most recent call last):
  File "mistake.py", line 34, in <module>
    {
  File "mistake.py", line 21, in mapper
    strange_number = 1.0 / (len(word) - 1)
ZeroDivisionError: float division by zero

End of LogType:stderr
***********************************************************************

Container: container_1643547082499_0005_01_000025 on rc1a-dataproc-d-9r5dg188d90q4f2a.mdb.yandexcloud.net_44115
LogAggregationType: AGGREGATED
LogType:

Идентификатор можно найти логах запуска, в интерфейсе или посмотреть список всех и найти там

In [50]:
! yarn application -list 

2022-01-30 14:15:27,368 INFO client.RMProxy: Connecting to ResourceManager at rc1a-dataproc-m-mk7wm6vzosv08ggu.mdb.yandexcloud.net/10.128.0.26:8032
2022-01-30 14:15:27,611 INFO client.AHSProxy: Connecting to Application History server at rc1a-dataproc-m-mk7wm6vzosv08ggu.mdb.yandexcloud.net/10.128.0.26:10200
Total number of applications (application-types: [], states: [SUBMITTED, ACCEPTED, RUNNING] and tags: []):0
                Application-Id	    Application-Name	    Application-Type	      User	     Queue	             State	       Final-State	       Progress	                       Tracking-URL


Чтобы заранее прикончить задачу, можно также использовать yarn

```bash
yarn application -kill <application_id>
```

### Переносим результаты

Все данные, с которыми мы работали только что, лежат на жестких дисках кластера и доступны только через hdfs

Если мы готовы презентовать наш результат миру, нужно его переместить в s3

In [52]:
! hdfs dfs -ls /user/tweets/top10-stop-words

Found 2 items
-rw-r--r--   1 ubuntu hadoop          0 2022-01-30 12:38 /user/tweets/top10-stop-words/_SUCCESS
-rw-r--r--   1 ubuntu hadoop        109 2022-01-30 12:38 /user/tweets/top10-stop-words/part-00000


In [53]:
! hdfs dfs -cp /user/tweets/top10-stop-words/part-00000 s3a://lsml2022alexius/top10.txt

2022-01-30 14:23:59,851 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2022-01-30 14:23:59,943 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2022-01-30 14:23:59,943 INFO impl.MetricsSystemImpl: s3a-file-system metrics system started
2022-01-30 14:24:04,635 INFO impl.MetricsSystemImpl: Stopping s3a-file-system metrics system...
2022-01-30 14:24:04,636 INFO impl.MetricsSystemImpl: s3a-file-system metrics system stopped.
2022-01-30 14:24:04,636 INFO impl.MetricsSystemImpl: s3a-file-system metrics system shutdown complete.


Можно проверять через интерфейс - файл оказался в бакете

### Hadoop жжет бабло

<img src="http://vostokovod.ru/assets/images/blog/2013/000333.png">

Полноценный кластер - весьма дорогое удовольствие, поэтому отлючайте его после использования. 